<a href="https://colab.research.google.com/github/Just-Aymz/Cafe-Sales/blob/main/Data_Processing/Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**
___

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno
from typing import Optional, Literal
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# **Read Files**
___

In [2]:
# Get the RAW url of the dataset file from repository
url = "https://raw.githubusercontent.com/Just-Aymz/Cafe-Sales/refs/heads/main/Data_Files/dirty_cafe_sales.csv"

# Read in the dataset using the url destination
df = pd.read_csv(url)

# **Data Identification**
___

In [3]:
# Return the shape and size of the dataset
shape = df.shape
size = df.size

print(f"The shape of the dataset is {shape}")
print(f"The size of the dataset is {size}")

The shape of the dataset is (10000, 8)
The size of the dataset is 80000


In [4]:
# Return the axes of the dataset
df.axes

[RangeIndex(start=0, stop=10000, step=1),
 Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
        'Payment Method', 'Location', 'Transaction Date'],
       dtype='object')]

In [5]:
# Return the data types of each features
df.dtypes

,0
Transaction ID,object
Item,object
Quantity,object
Price Per Unit,object
Total Spent,object
Payment Method,object
Location,object
Transaction Date,object


In [6]:
# Return overview of the first 5 rows of the dataset
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


# **Dataset Preprocessing**
___

# 1. **Miscellaneous Errors**

## 1.1 **Columns**

In [7]:
# Make a copy of the current datafame
df_2 = df.copy()

# Replace the spaces in the column names with an underscore
df_2.columns = df.columns.str.replace(' ', '_')

## 1.2 **Index**

In [8]:
# Make a copy of the current datafame
df_3 = df_2.copy()

# Make the transcation id feature the index
df_3.set_index('Transaction_ID', inplace=True)

In [9]:
# Confirm the changes
df_3.head(1)

,Item,Quantity,Price_Per_Unit,Total_Spent,Payment_Method,Location,Transaction_Date
Transaction_ID,,,,,,,
TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08


## 1.3 **Data Errors**

In [10]:
for col in df_3.columns.drop('Transaction_Date'):
  print(f"{col}: {df_3[col].unique()}\n")

Item: ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' nan
 'ERROR' 'Juice' 'Tea']

Quantity: ['2' '4' '5' '3' '1' 'ERROR' 'UNKNOWN' nan]

Price_Per_Unit: ['2.0' '3.0' '1.0' '5.0' '4.0' '1.5' nan 'ERROR' 'UNKNOWN']

Total_Spent: ['4.0' '12.0' 'ERROR' '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0' '5.0'
 '3.0' '6.0' nan 'UNKNOWN' '2.0' '1.0' '7.5' '4.5' '1.5']

Payment_Method: ['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR' nan]

Location: ['Takeaway' 'In-store' 'UNKNOWN' nan 'ERROR']



In [11]:
# Make a copy of the current datafame
df_4 = df_3.copy()

# Replace all the 'UNKNOWN' and the 'ERROR' values with np.nan
df_4.replace(['UNKNOWN', 'ERROR'], np.nan, inplace=True)

## 1.3 **Feature Casting**

In [12]:
# Make a copy of the current datafame
df_5 = df_4.copy()

# Cast the features to the correct features
df_5 = df_5.astype(
    {'Quantity': 'float64',
     'Total_Spent': 'float64',
     'Price_Per_Unit': 'float64',
     'Transaction_Date': 'datetime64[ns]'}
)

In [13]:
df_5.dtypes

,0
Item,object
Quantity,float64
Price_Per_Unit,float64
Total_Spent,float64
Payment_Method,object
Location,object
Transaction_Date,datetime64[ns]


## 2. **Data Cleaning**

In [14]:
def data_cleaning(
    dataframe: pd.DataFrame, operation: Literal['duplicate', 'nulls']
) -> None:
  """
  Identifies and reports the number of duplicate or null values in a given
  DataFrame.

  Parameters:
  - dataframe (pd.DataFrame): The DataFrame to analyze.
  - operation (Literal['duplicate', 'nulls']): Specifies the type of check to
  perform.
      - 'duplicate': Counts and reports duplicate rows.
      - 'nulls': Counts and reports null (NaN) values.

  Returns:
  - None: Prints the count of detected duplicate or null values.
    """
  if operation == 'duplicate':
    dupes = dataframe.duplicated().sum()
    if dupes > 0:
      print(f"{dupes} duplicate values found")
    elif dupes == 0:
      print(f"{dupes} duplicate values found")
  else:
    nulls = np.count_nonzero(dataframe.isnull().values)
    if nulls > 0:
      print(f"{nulls} null values found")
    elif nulls == 0:
      print(f"{nulls} null values found")


### 2.1 **Duplicate Values**

In [15]:
# Return the total number of duplicates in the dataset
data_cleaning(df_5, 'duplicate')

255 duplicate values found


### 2.2 **Null Values**

In [16]:
# Return the total number of null values in the dataset
data_cleaning(df_5, 'nulls')

10082 null values found


In [17]:
# Return the total null values per feature
df_5.isnull().sum()

,0
Item,969
Quantity,479
Price_Per_Unit,533
Total_Spent,502
Payment_Method,3178
Location,3961
Transaction_Date,460


In [18]:
# Return a subset where the Transaction Date feature contains null values
df_5[df_5['Transaction_Date'].isnull().values].head()

,Item,Quantity,Price_Per_Unit,Total_Spent,Payment_Method,Location,Transaction_Date
Transaction_ID,,,,,,,
TXN_3051279,Sandwich,2.0,4.0,8.0,Credit Card,Takeaway,NaT
TXN_7640952,Cake,4.0,3.0,12.0,Digital Wallet,Takeaway,NaT
TXN_7710508,NaN,5.0,1.0,5.0,Cash,NaN,NaT
TXN_2091733,Salad,1.0,5.0,5.0,NaN,In-store,NaT
TXN_7028009,Cake,4.0,3.0,12.0,NaN,Takeaway,NaT


In [19]:
"""
Since the Transactional ID uniquely identifies each transaction and does not
provide any information about the transaction date—and none of the other
features offer sufficient context to deduce this information—any records with a
null value in the transactional date column must be removed.
"""
# Create a copy of the current dataframe
df_6 = df_5.copy()

# Drop the null values from the transaction date feature
df_6.dropna(subset=['Transaction_Date'], axis=0, inplace=True)


In [20]:
# Check the data has changed
df_6.isnull().sum()

,0
Item,927
Quantity,454
Price_Per_Unit,506
Total_Spent,476
Payment_Method,3015
Location,3779
Transaction_Date,0


In [21]:
"""
Use KNNImputation to impute the missing categoticsal features
Step 1: Identify the categorical features
Step 2: Use label encoding on the categorical features
Step 3: Use KNNImputation on the entire dataframe
Step 4: Convert the array back into a dataframe
Step 5: Invert back into string values
"""
# Create a copy of the current dataframe
df_7 = df_6.copy()

def category_encoding(
    dataframe: pd.DataFrame, features: list[str]
) -> pd.DataFrame:
  """
  This function performs category encoding on categorical features using
  Label Encoding, applies KNN imputation to fill missing values, and then
  converts encoded features back to their original categorical values.

  Parameters:
      dataframe (pd.DataFrame): The input dataframe containing the data with
                                missing values.
      features (list[str]): A list of column names corresponding to categorical
                            features
                            that need to be encoded and imputed.

  Returns:
      pd.DataFrame: The dataframe with imputed categorical features restored
                    to their original values.
  """

  # Initialize a dictionary to store label encoders for each feature
  label_encoders = {}

  for feature in features:
    label_encoder = LabelEncoder()
    dataframe[feature] = label_encoder.fit_transform(dataframe[feature])
    label_encoders[feature] = label_encoder # Save encoder for later decoding

  # Separate datetime columns to avoid dtype issues
  datetime_cols = dataframe.select_dtypes(include=['datetime64']).columns
  datetime_df = dataframe[datetime_cols].copy()  # Preserve datetime columns
  dataframe = (
      dataframe.drop(columns=datetime_cols)  # Drop datetime columns imputation
  )

  def knn_Imputer(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    A function that instantiates an instance of the KNNImputer, which is used
    to fit and transform the dataframe passed as an argument. This function
    imputes missing values based on the K-nearest neighbors algorithm and
    returns the imputed dataframe.

    Parameters:
        dataframe (pd.DataFrame): The dataframe containing missing values that
        need to be imputed.

    Returns:
        pd.DataFrame: The dataframe with imputed values for the specified
        features.
    """
    imputer = KNNImputer(n_neighbors=3) # Instantiate the imputer
    imputed_data = imputer.fit_transform(dataframe)

    # Use the column sequence of the original dataframe
    imputed_df = pd.DataFrame(imputed_data, columns=dataframe.columns)
    print(imputed_df.head())

    return imputed_df

  imputed_df = (
      knn_Imputer(dataframe)  # Store the imputed dataframe as a variable
  )

  for col in dataframe.columns:
    if col in label_encoders:  # Only apply inverse_transform to categorical features
        imputed_df[col] = imputed_df[col].astype(int).round(0)
        imputed_df[col] = label_encoders[col].inverse_transform(imputed_df[col])
    else:
        imputed_df[col] = imputed_df[col]  # Keep numerical columns as they are

  # Merge datetime columns back
  imputed_df = pd.concat([imputed_df, datetime_df], axis=1)

  return imputed_df

In [25]:
# Store the categorical features of the dataframe
categorical_features = df_6.select_dtypes(include='object').columns
print(categorical_features)


# Return the imputed dataframe
df_8 = category_encoding(df_7, categorical_features)

Index(['Item', 'Payment_Method', 'Location'], dtype='object')
   Item  Quantity  Price_Per_Unit  Total_Spent  Payment_Method  Location
0   1.0       2.0             2.0     4.000000             1.0       1.0
1   0.0       4.0             3.0    12.000000             0.0       0.0
2   2.0       4.0             1.0     3.333333             1.0       0.0
3   4.0       2.0             5.0    10.000000             3.0       2.0
4   1.0       2.0             2.0     4.000000             2.0       0.0


In [26]:
df_8.head()

,Item,Quantity,Price_Per_Unit,Total_Spent,Payment_Method,Location,Transaction_Date
0,Coffee,2.0,2.0,4.000000,Credit Card,Takeaway,NaT
1,Cake,4.0,3.0,12.000000,Cash,In-store,NaT
2,Cookie,4.0,1.0,3.333333,Credit Card,In-store,NaT
3,Salad,2.0,5.0,10.000000,NaN,NaN,NaT
4,Coffee,2.0,2.0,4.000000,Digital Wallet,In-store,NaT


In [27]:
categorical_features

Index(['Item', 'Payment_Method', 'Location'], dtype='object')

In [28]:
df_8.isnull().sum()

,0
Item,10467
Quantity,9540
Price_Per_Unit,9540
Total_Spent,9540
Payment_Method,12555
Location,13319
Transaction_Date,9540
